# Model dengan Fitur TF-IDF

File ini berisikan program untuk pembangunan model menggunakan fitur tf-idf.
Prediksi dilakukan terhadap label fakultas dan label rumpun.
Eksperimen terhadap fitur bag of words menyangkut preprocessing, metode pembangunan model, dan jumlah label.

### Load Data

In [25]:
from nltk.corpus import stopwords
from nltk import stem

In [26]:
dataset = open('korpus-ver-2.txt', 'r', encoding="utf-8")
lines = dataset.readlines()
len_lines = len(lines)
counter = 0
list_kalimat = []
list_rumpun = []
list_fakultas = []


for i in range(0,len_lines,3):
    line_i = lines[i]
    judul = line_i
    
    judul = judul[8:-9] #buang tag <JUDUL> dari kalimat
   
    sinopsis = lines[i+1]
    sinopsis = sinopsis[10:-13] #buang tag <SINOPSIS> dari kalimat
   
    
    kalimat = judul + " " + sinopsis


    fakultas_arr = lines[i+2].lower().split()
    fakultas = fakultas_arr[1]
    
    
    if(fakultas=="rik"):
        label = "rik"
    elif(fakultas=="fmipa"):
        label = "saintek"
    elif(fakultas=="fasilkom"):
        label = "saintek"
    elif(fakultas=="ft"):
        label = "saintek"
    elif(fakultas=="fisip"):
        label = "soshum"
    elif(fakultas=="fib"):
        label = "soshum"
    elif(fakultas=="fh"):
        label = "soshum"
    elif(fakultas=="feb"):
        label = "soshum"
    elif(fakultas=="psikologi"):
        label = "soshum"
    
    list_kalimat.append(kalimat)
    list_fakultas.append(fakultas)
    list_rumpun.append(label)

### Preprocessing

In [27]:
N = len(list_kalimat)
list_cleaned = []
list_stemmed = []
list_lemmatized = []

stemmer = stem.snowball.SnowballStemmer("english")
stop_words = stopwords.words('english')
whitelist = set('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')

for i in range(N):
    kalimat = list_kalimat[i]
    
    #lowercase
    kalimat = kalimat.lower()
    
    #stopwords and tandabaca removal
    words = kalimat.split(" ")
    kalimat_clean_arr = [''.join(filter(whitelist.__contains__, word)) for word in words if word not in stop_words]
    
    #stemming
    kalimat_stemmed = ' '.join(stemmer.stem(w) for w in kalimat_clean_arr)
    
    # TODO lematisasi
    kalimat_lemma = kalimat_stemmed
    
    kalimat_cleaned = ' '.join(kalimat_clean_arr)
    
    list_cleaned.append(kalimat_cleaned)
    list_stemmed.append(kalimat_stemmed)
    list_lemmatized.append(kalimat_lemma)    
    
    
# print(list_cleaned[0])
# print()
# print(list_stemmed[0])
# print()
# print(list_lemmatized[0])


### Ekstraksi Fitur data

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer1 = TfidfVectorizer(min_df=0, lowercase=False)
vectorizer1.fit(list_cleaned)

X1 = vectorizer1.transform(list_cleaned).toarray()

vectorizer2 = TfidfVectorizer(min_df=0, lowercase=False)
vectorizer2.fit(list_stemmed)

X2 = vectorizer2.transform(list_stemmed).toarray()

vectorizer3 = TfidfVectorizer(min_df=0, lowercase=False)
vectorizer3.fit(list_lemmatized)

X3 = vectorizer3.transform(list_lemmatized).toarray()


yf = list_fakultas
yr = list_rumpun

### Split Data test

In [29]:
from sklearn.model_selection import train_test_split
ind = [i for i in range(N)]

ind_train,ind_test, _ , _ = train_test_split(ind, ind, test_size=0.25, random_state=1000)


X_train1 = []
X_test1 = []
X_train2 = []
X_test2 = []
X_train3 = []
X_test3 = []

y_train1 = []
y_test1 = []

y_train2 = []
y_test2 = []


for i in ind_train:
#     X_train1.append(list_cleaned[i])
#     X_train2.append(list_stemmed[i])
#     X_train3.append(list_lemmatized[i])
    X_train1.append(X1[i])
    X_train2.append(X2[i])
    X_train3.append(X3[i])
    
    y_train1.append(yf[i])
    y_train2.append(yr[i])
    

for i in ind_test:
    X_test1.append(X1[i])
    X_test2.append(X2[i])
    X_test3.append(X3[i])
    
    y_test1.append(yf[i])
    y_test2.append(yr[i])
    
print(X_test1[1])
print()
print(X_test2[1])
print()
print(y_test1[1])
print(y_test2[1])

[0. 0. 0. ... 0. 0. 0.]

[0. 0. 0. ... 0. 0. 0.]

fib
soshum


### Pembangunan Model dan Akurasi

#### Fungsi untuk confusion matrix

In [30]:
import itertools
import numpy
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes, normalize=False,  title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    plt.colorbar()
    tick_marks = numpy.arange(len(classes)) 
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix") 
    else:
        print('Confusion matrix, without normalization')
    #print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])): 
        plt.text(j, i, cm[i, j],  horizontalalignment="center",  color="white" if cm[i, j] > thresh else "black") 

    plt.tight_layout() 
    plt.ylabel('True label') 
    plt.xlabel('Predicted label')

#### Log Regression

In [31]:
from sklearn.linear_model import LogisticRegression

classifier_log = LogisticRegression()

classifier_log.fit(X_train1, y_train1)
predicted = classifier_log.predict(X_test1)
score = classifier_log.score(X_test1, y_test1)

print("Logistic Regression untuk X1 dan Y1 :", score)

# matrix = confusion_matrix(y_test1, predicted)
# plot_confusion_matrix(matrix, classes = set(yf), title= 'CM for X1 and Y1')

classifier_log.fit(X_train2, y_train1)
score = classifier_log.score(X_test2, y_test1)

print("Logistic Regression untuk X2 dan Y1 :", score)

classifier_log.fit(X_train3, y_train1)
score = classifier_log.score(X_test3, y_test1)

print("Logistic Regression untuk X3 dan Y1 :", score)


classifier_log.fit(X_train1, y_train2)
predicted = classifier_log.predict(X_test1)
score = classifier_log.score(X_test1, y_test2)

print("Logistic Regression untuk X1 dan Y2 :", score)

# matrix = confusion_matrix(y_test2, predicted)
# plot_confusion_matrix(matrix, classes = set(yr), title= 'CM for X1 and Y2')

classifier_log.fit(X_train2, y_train2)
score = classifier_log.score(X_test2, y_test2)

print("Logistic Regression untuk X2 dan Y2 :", score)

classifier_log.fit(X_train3, y_train2)
score = classifier_log.score(X_test3, y_test2)

print("Logistic Regression untuk X3 dan Y2 :", score)

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression untuk X1 dan Y1 : 0.2698412698412698
Logistic Regression untuk X2 dan Y1 : 0.30158730158730157


C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression untuk X3 dan Y1 : 0.30158730158730157
Logistic Regression untuk X1 dan Y2 : 0.6190476190476191
Logistic Regression untuk X2 dan Y2 : 0.6507936507936508
Logistic Regression untuk X3 dan Y2 : 0.6507936507936508


C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver 

#### Gaussian Naive Bayes

In [32]:
from sklearn.naive_bayes import GaussianNB
classifier_gnb = GaussianNB()

classifier_gnb.fit(X_train1, y_train1)
score = classifier_gnb.score(X_test1, y_test1)

print("Gaussian NB untuk X1 dan Y1 :", score)

classifier_gnb.fit(X_train2, y_train1)
score = classifier_gnb.score(X_test2, y_test1)

print("Gaussian NB untuk X2 dan Y1 :", score)

classifier_gnb.fit(X_train3, y_train1)
score = classifier_gnb.score(X_test3, y_test1)

print("Gaussian NB untuk X3 dan Y1 :", score)

classifier_gnb.fit(X_train1, y_train2)
score = classifier_gnb.score(X_test1, y_test2)

print("Gaussian NB untuk X1 dan Y2 :", score)

classifier_gnb.fit(X_train2, y_train2)
score = classifier_gnb.score(X_test2, y_test2)

print("Gaussian NB untuk X2 dan Y2 :", score)

classifier_gnb.fit(X_train3, y_train2)
score = classifier_gnb.score(X_test3, y_test2)

print("Gaussian NB untuk X3 dan Y2 :", score)

Gaussian NB untuk X1 dan Y1 : 0.3968253968253968
Gaussian NB untuk X2 dan Y1 : 0.4126984126984127
Gaussian NB untuk X3 dan Y1 : 0.4126984126984127
Gaussian NB untuk X1 dan Y2 : 0.7777777777777778
Gaussian NB untuk X2 dan Y2 : 0.8253968253968254
Gaussian NB untuk X3 dan Y2 : 0.8253968253968254


#### Multinomial Naive Bayes

In [33]:
from sklearn.naive_bayes import MultinomialNB
classifier_mnb = MultinomialNB()

classifier_mnb.fit(X_train1, y_train1)
score = classifier_mnb.score(X_test1, y_test1)

print("Multinomial NB untuk X1 dan Y1 :", score)

classifier_mnb.fit(X_train2, y_train1)
score = classifier_mnb.score(X_test2, y_test1)

print("Multinomial NB untuk X2 dan Y1 :", score)

classifier_mnb.fit(X_train3, y_train1)
score = classifier_mnb.score(X_test3, y_test1)

print("Multinomial NB untuk X3 dan Y1 :", score)

classifier_mnb.fit(X_train1, y_train2)
score = classifier_mnb.score(X_test1, y_test2)

print("Multinomial NB untuk X1 dan Y2 :", score)

classifier_mnb.fit(X_train2, y_train2)
score = classifier_mnb.score(X_test2, y_test2)

print("Multinomial NB untuk X2 dan Y2 :", score)

classifier_mnb.fit(X_train3, y_train2)
score = classifier_mnb.score(X_test3, y_test2)

print("Multinomial NB untuk X3 dan Y2 :", score)

Multinomial NB untuk X1 dan Y1 : 0.2222222222222222
Multinomial NB untuk X2 dan Y1 : 0.2222222222222222
Multinomial NB untuk X3 dan Y1 : 0.2222222222222222
Multinomial NB untuk X1 dan Y2 : 0.6190476190476191
Multinomial NB untuk X2 dan Y2 : 0.6190476190476191
Multinomial NB untuk X3 dan Y2 : 0.6190476190476191


#### Random Forest

In [34]:
from sklearn.ensemble import RandomForestClassifier


classifier_rf = RandomForestClassifier()

classifier_rf.fit(X_train1, y_train1)
score = classifier_rf.score(X_test1, y_test1)

print("Random Forest untuk X1 dan Y1 :", score)

classifier_rf.fit(X_train2, y_train1)
score = classifier_rf.score(X_test2, y_test1)

print("Random Forest untuk X2 dan Y1 :", score)

classifier_rf.fit(X_train3, y_train1)
score = classifier_rf.score(X_test3, y_test1)

print("Random Forest untuk X3 dan Y1 :", score)

classifier_rf.fit(X_train1, y_train2)
score = classifier_rf.score(X_test1, y_test2)

print("Random Forest untuk X1 dan Y2 :", score)

classifier_rf.fit(X_train2, y_train2)
score = classifier_rf.score(X_test2, y_test2)

print("Random Forest untuk X2 dan Y2 :", score)

classifier_rf.fit(X_train3, y_train2)
score = classifier_rf.score(X_test3, y_test2)

print("Random Forest untuk X3 dan Y2 :", score)

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random Forest untuk X1 dan Y1 : 0.30158730158730157
Random Forest untuk X2 dan Y1 : 0.23809523809523808
Random Forest untuk X3 dan Y1 : 0.36507936507936506
Random Forest untuk X1 dan Y2 : 0.6190476190476191
Random Forest untuk X2 dan Y2 : 0.6507936507936508
Random Forest untuk X3 dan Y2 : 0.6666666666666666


#### AdaBoost

In [35]:
from sklearn.ensemble import AdaBoostClassifier

classifier_ada = AdaBoostClassifier()

classifier_ada.fit(X_train1, y_train1)
score = classifier_ada.score(X_test1, y_test1)

print("AdaBoost untuk X1 dan Y1 :", score)

classifier_ada.fit(X_train2, y_train1)
score = classifier_ada.score(X_test2, y_test1)

print("AdaBoost untuk X2 dan Y1 :", score)

classifier_ada.fit(X_train3, y_train1)
score = classifier_ada.score(X_test3, y_test1)

print("AdaBoost untuk X3 dan Y1 :", score)

classifier_ada.fit(X_train1, y_train2)
score = classifier_ada.score(X_test1, y_test2)

print("AdaBoost untuk X1 dan Y2 :", score)

classifier_ada.fit(X_train2, y_train2)
score = classifier_ada.score(X_test2, y_test2)

print("AdaBoost untuk X2 dan Y2 :", score)

classifier_ada.fit(X_train3, y_train2)
score = classifier_ada.score(X_test3, y_test2)

print("AdaBoost untuk X3 dan Y2 :", score)

AdaBoost untuk X1 dan Y1 : 0.12698412698412698
AdaBoost untuk X2 dan Y1 : 0.12698412698412698
AdaBoost untuk X3 dan Y1 : 0.12698412698412698
AdaBoost untuk X1 dan Y2 : 0.6507936507936508
AdaBoost untuk X2 dan Y2 : 0.6349206349206349
AdaBoost untuk X3 dan Y2 : 0.6349206349206349


#### Decision Tree

In [36]:
from sklearn.tree import DecisionTreeClassifier

classifier_dt = DecisionTreeClassifier()

classifier_dt.fit(X_train1, y_train1)
score = classifier_dt.score(X_test1, y_test1)

print("Decision Tree untuk X1 dan Y1 :", score)

classifier_dt.fit(X_train2, y_train1)
score = classifier_dt.score(X_test2, y_test1)

print("Decision Tree untuk X2 dan Y1 :", score)

classifier_dt.fit(X_train3, y_train1)
score = classifier_dt.score(X_test3, y_test1)

print("Decision Tree untuk X3 dan Y1 :", score)

classifier_dt.fit(X_train1, y_train2)
score = classifier_dt.score(X_test1, y_test2)

print("Decision Tree untuk X1 dan Y2 :", score)

classifier_dt.fit(X_train2, y_train2)
score = classifier_dt.score(X_test2, y_test2)

print("Decision Tree untuk X2 dan Y2 :", score)

classifier_dt.fit(X_train3, y_train2)
score = classifier_dt.score(X_test3, y_test2)

print("Decision Tree untuk X3 dan Y2 :", score)

Decision Tree untuk X1 dan Y1 : 0.2698412698412698
Decision Tree untuk X2 dan Y1 : 0.2857142857142857
Decision Tree untuk X3 dan Y1 : 0.2857142857142857
Decision Tree untuk X1 dan Y2 : 0.746031746031746
Decision Tree untuk X2 dan Y2 : 0.8095238095238095
Decision Tree untuk X3 dan Y2 : 0.7777777777777778
